Questo notebook per Colaboratory di Google mostra come utilizzando i modelli di **Helsinki-NLP** per tradurre del testo

In [ ]:
!pip install transformers sentencepiece sacremoses fasttext

import os
if not os.path.isfile("/content/lid218e.bin"):
  !wget https://dl.fbaipublicfiles.com/nllb/lid/lid218e.bin

Iniziamo inserendo il testo da tradurre

In [ ]:
testo_src = input("Inserisci il testo da tradurre: ")

Infine inseriamo una o più parole nella lingua di testo per la traduzione

In [ ]:
testo_dst = input("Inserisci una o più parole nella lingua per la traduziobne, es. name: ")

Cominciamo a rilevare le lingue usate per la traduzione per il nome del modello da usare

In [ ]:
import fasttext

plm = "/content/lid218e.bin"
model = fasttext.load_model(plm)

p = model.predict(testo_src, k=-1, threshold=0.5)
input_lang = p[0][0].replace('__label__', '')
print("Lingua sorgente: " + input_lang)

p = model.predict(testo_dst, k=-1, threshold=0.5)
output_lang = p[0][0].replace('__label__', '')
print("Lingua destinaizone: " + output_lang)

src = input_lang[:2]
trg = output_lang[:2]
model_name = f'Helsinki-NLP/opus-mt-{src}-{trg}'

Individua la presenza di una GPU

In [ ]:
import torch

if not torch.cuda.is_available():
  print('WARNING: You may want to change the runtime to GPU for faster training!')
  device = 'cpu'
else:
  device = 'cuda:0'

Infine traduciamo il testo nella lingua desiderata

Esempio n°1

In [ ]:
from transformers import MarianTokenizer, MarianMTModel
from typing import List

tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name).to(device)
batch = tokenizer([testo_src], return_tensors="pt", padding=True).to(device)
translated = model.generate(**batch)

for t in translated:
    print(tokenizer.decode(t, skip_special_tokens=True))

Esempio n°2

In [ ]:
from transformers import pipeline

pipe = pipeline("translation", model=model_name,device=device)
print(pipe(testo_src))